In [7]:
### 测试stationary函数
import os
from util import stationary
import numpy as np
# 创建一个简单的过渡矩阵
A = np.array([[0.4, 0.6],
              [0.2, 0.8]])

# 调用函数
result = stationary(A)
result

/root/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1272: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


array([0.5, 1.5])

In [1]:
import os
import numpy as np
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from scipy.sparse import csr_matrix
import networkx as nx

from scipy.linalg import orthogonal_procrustes

import torch
import torch.optim as optim
from torch_geometric.data import Data

from sklearn.manifold import TSNE
from sklearn.datasets import make_moons  # 导入make_moons函数

from util import Net, GIN, GAT, stationary, reconstruct, dG


np.random.seed(0)
torch.manual_seed(0)

n = 2000
m = 500
DISTANCE = 0.1

# 导入moon数据集
x, y = make_moons(n_samples=n, noise=0.1, random_state=0) 
# 分割训练集和测试机
n_train = int(n * 0.7)
train_ind = torch.randperm(n)[:n_train]
test_ind = torch.LongTensor(list(set(np.arange(n)) - set(train_ind.tolist())))
# K = int(np.sqrt(n) * np.log2(n) / 10) #作为隐藏输入
D = pairwise_distances(x)# 邻接距离矩阵
# 二值化，得到邻接矩阵
A_binary = np.where(D <= DISTANCE, 1, 0)
# A_binary
# 获得所有的边，二元组格式
row_indices, col_indices = np.where(A_binary == 1)
edges = list(zip(row_indices, col_indices))

# 转为tensor
x = torch.tensor(x, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()
data = Data(x=x, y=y, edge_index=edges)
print(data)

# 喂到GIN网络中
net = GIN(m)
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.train()
for epoch in range(2):
    ind = torch.eye(n)[:, torch.randperm(n)[:m]]
    # 将节点选择矩阵与特征矩阵连接在一起
    X_extended = torch.hstack([x, ind]) # 
    print("拼接后的矩阵shape：", X_extended.shape)
    data = Data(x=X_extended, edge_index=edges)
    print(data)
    rec = net(data) # reconstruct
    loss = dG(x[train_ind], rec[train_ind]) #train loss
    print(f'Epoch {epoch + 1}, Loss: {float(loss)}')
    optimizer.zero_grad() # zero gradient
    loss.backward()
    optimizer.step()

# R, _ = orthogonal_procrustes(x.numpy(), rec.detach().numpy())
# rec_GIN = rec.detach().numpy() @ R.T
# loss_GIN = float(dG(x, rec))
# # 创建图形
# fig, ax = plt.subplots(1, 2, figsize=(14, 4))
# # ax两个画布
# ax[0].scatter(x[:, 0], x[:, 1], c=y, s=10, cmap='viridis', rasterized=True)
# ax[0].set_xticks([])
# ax[0].set_yticks([])
# ax[0].set_facecolor('#eeeeee')
# txt = ax[0].text(0.05, 0.05, 'Ground Truth', color='k', fontsize=14, weight='bold', transform=ax[0].transAxes)
# txt.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='#eeeeee')])

# # 绘制 GIN 网络的重构结果
# ax[1].scatter(rec_GIN[:, 0], rec_GIN[:, 1], c=y, s=10, cmap='viridis', rasterized=True)
# ax[1].set_xticks([])
# ax[1].set_yticks([])
# txt = ax[1].text(0.05, 0.05, f'GIN $d_G = {loss_GIN:.2f}$', color='k', fontsize=14, weight='bold', transform=ax[1].transAxes)
# txt.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])

# fig.subplots_adjust()

# if not os.path.exists('imgs'):
#     os.mkdir('imgs')

# fig.savefig('imgs/semi_moon_GIN.png', bbox_inches='tight', dpi=300)

# plt.show()

Data(x=[2000, 2], edge_index=[2, 53372], y=[2000])
拼接后的矩阵shape： torch.Size([2000, 502])
Data(x=[2000, 502], edge_index=[2, 53372])
torch.Size([2, 2]) torch.Size([2]) torch.Size([2, 2])
tensor([1.9461e+01, 8.3291e-04], grad_fn=<LinalgSvdBackward0>)
Epoch 1, Loss: 1.2955381870269775
拼接后的矩阵shape： torch.Size([2000, 502])
Data(x=[2000, 502], edge_index=[2, 53372])
torch.Size([2, 2]) torch.Size([2]) torch.Size([2, 2])
tensor([2.8864e+01, 1.9608e-03], grad_fn=<LinalgSvdBackward0>)
Epoch 2, Loss: 1.282875418663025


In [15]:
import torch
n=10
m=5
rc_list = torch.randperm(n)[:m]
ind = torch.eye(n)[:, rc_list]
print(rc_list)
ind

tensor([2, 6, 9, 7, 1])


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [9]:
import gensim

def main():
    sparse_version =gensim.__version__
    print("torch-sparse 版本为:", sparse_version)

if __name__ == "__main__":
    main()


torch-sparse 版本为: 4.3.2


In [3]:
# reconstruct
from util import reconstruct
import numpy as np
# 构造测试数据
K = 10
n = 20
pr = np.random.rand(n)
print(pr)
m = 5
fr = np.random.randint(0, n, size=K)
to = np.random.randint(0, n, size=K)
for i,j in zip(fr, to):
    print(f"{i}->{j}", end='\t')

# 调用reconstruct函数
reconstructed_data = reconstruct(K, pr, n, m, fr, to)

# 打印重构后的数据
print("Reconstructed Data:")
print(reconstructed_data)

[0.10853552 0.42812706 0.07912775 0.82336341 0.48571637 0.67106305
 0.16808549 0.67425444 0.00458491 0.0531815  0.3954835  0.14645842
 0.83055454 0.0173432  0.15689277 0.64562884 0.89534729 0.09211132
 0.30262194 0.16781474]
10->5	12->3	18->15	3->8	1->8	18->12	18->13	6->7	18->11	1->13	

ValueError: row, column, and data array must all be the same length